# Abstract

# Import Functions

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
import os
import sys

# Build an absolute path from this notebook's parent directory
module_path = os.path.abspath('../')

# Add to sys.path if not already present
if module_path not in sys.path:
    sys.path.append(module_path)

# Now you can import the desired function or class
from Functions.seaborn_plotting_function import seaborn_plotter
from Functions.basic_functions import save_pickle
#from Functions.data_transformation_pipeline import Transformation_Pipeline

# Logistics

In [3]:
raw_database_folder = '../Database/raw'
mart_database_folder = '../Database/marts'

In [4]:
plotter = seaborn_plotter()

# load files

In [5]:
raw_db = {}

In [6]:

for file in os.listdir(raw_database_folder):
    if '.csv' in file:
        key = file.replace('.csv', '')
        raw_db[key] = pd.read_csv(os.path.join(raw_database_folder, file))
        raw_db[key] = raw_db[key].convert_dtypes()

raw_db['contact_log']['date'] = pd.to_datetime(raw_db['contact_log']['date'])
raw_db['members']['member_join_date'] = pd.to_datetime(raw_db['members']['member_join_date'])

In [7]:
for key in raw_db:
    print(key)

contact_log
members
member_contacts
type_of_contact


# Step 1: ETL process (would want to move this to DBT or something of that sort to do an ETL process using an orchestration tool)

In [8]:
member_contract_columns = ['connection_id', 'member_id', 'contact_id', 'local']
contact_log_columns = ['member_id', 'contact_id', 'date', 'type_of_contact']
members_columns = ['member_id', 'member_join_date']

## Step 1A: get connection id most recent contact

In [9]:
raw_db['contact_log']['most_recent_contact'] = raw_db['contact_log'].groupby(['member_id', 'contact_id'])['date'].rank(method='max', ascending=False)
raw_db['contact_log']['most_recent_contact'] = raw_db['contact_log']['most_recent_contact'].astype(int)
recent_contacts = raw_db['contact_log'][raw_db['contact_log']['most_recent_contact'] == 1]

## Step 1B: get contact_member_log

In [10]:
contact_member_logs = pd.merge(raw_db['member_contacts'].filter(member_contract_columns), recent_contacts.filter(contact_log_columns), how='inner', on = ['member_id', 'contact_id'])
contact_member_logs = contact_member_logs.rename(columns= {'date': 'most_recent_contact'})

In [11]:
no_contact_yet = raw_db['member_contacts'][~raw_db['member_contacts']['connection_id'].isin(contact_member_logs['connection_id'].unique())]
no_contact_yet = no_contact_yet.filter(member_contract_columns)

In [12]:
no_contact_yet = pd.merge(no_contact_yet, raw_db['members'].filter(members_columns), how='inner', on='member_id')
no_contact_yet['type_of_contact'] = 0
no_contact_yet = no_contact_yet.rename(columns= {'member_join_date': 'most_recent_contact'})

## Step 1C: Additional Metrics

In [13]:
number_of_contacts = raw_db['contact_log'].groupby(['member_id', 'contact_id']).count()
number_of_contacts = number_of_contacts.filter(['date'])
number_of_contacts = number_of_contacts.rename(columns = {'date': 'number_of_contacts'})
number_of_contacts = number_of_contacts.reset_index(drop = False)

In [14]:
contact_member_logs = pd.concat([contact_member_logs, no_contact_yet])
contact_member_logs['days_since_contact'] = dt.date.today()
contact_member_logs['days_since_contact'] = pd.to_datetime(contact_member_logs['days_since_contact'])
contact_member_logs['days_since_contact'] = (contact_member_logs['days_since_contact'] - contact_member_logs['most_recent_contact']).dt.days

In [15]:
contact_member_logs = pd.merge(contact_member_logs, number_of_contacts, how = 'left', on = ['member_id', 'contact_id'])
contact_member_logs = contact_member_logs.fillna(0)
contact_member_logs['number_of_contacts'] = contact_member_logs['number_of_contacts'].astype(int)
contact_member_logs['type_of_contact'] = contact_member_logs['type_of_contact'].astype(int)
contact_member_logs

,connection_id,member_id,contact_id,local,most_recent_contact,type_of_contact,days_since_contact,number_of_contacts
0,22,1,22,0,2025-06-26,1,203,2
1,34,1,34,1,2025-06-24,4,205,1
2,35,1,35,1,2025-06-24,4,205,1
3,52,1,52,0,2025-06-26,1,203,1
4,1,1,1,0,2025-06-10,0,219,0
...,...,...,...,...,...,...,...,...
80,81,1,81,1,2025-06-10,0,219,0
81,82,1,82,1,2025-06-10,0,219,0
82,83,1,83,0,2025-06-10,0,219,0
83,84,1,84,0,2025-06-10,0,219,0


In [16]:
contact_member_logs.to_csv(os.path.join(mart_database_folder, 'contact_member_logs.csv'))

# Step 2: normalize data for local and non local data

In [17]:
normalize_columns = ['type_of_contact', 'days_since_contact', 'number_of_contacts']

In [18]:
contact_member_logs['type_of_contact'].unique()

array([1, 4, 0])

In [19]:
etl_data = {'local': contact_member_logs[contact_member_logs['local'] == 1],
            'non_local': contact_member_logs[contact_member_logs['local'] == 0],
           }

In [20]:
for data_type in ('local', 'non_local'):
    for column_name in normalize_columns:
        scaler = MinMaxScaler()
        etl_data[data_type][f"norm_{column_name}"] = scaler.fit_transform(etl_data[data_type][[column_name]].values)

    for column_name in normalize_columns:
        if column_name != 'days_since_contact':
            etl_data[data_type][f"inv_norm_{column_name}"] = 1 - etl_data[data_type][f"norm_{column_name}"]
    etl_data[data_type]['score'] = etl_data[data_type][['inv_norm_type_of_contact', 'norm_days_since_contact', 'inv_norm_number_of_contacts']].mean(axis=1)


C:\Users\alber\AppData\Local\Temp\ipykernel_31652\1692067836.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  etl_data[data_type][f"norm_{column_name}"] = scaler.fit_transform(etl_data[data_type][[column_name]].values)
C:\Users\alber\AppData\Local\Temp\ipykernel_31652\1692067836.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  etl_data[data_type][f"norm_{column_name}"] = scaler.fit_transform(etl_data[data_type][[column_name]].values)
C:\Users\alber\AppData\Local\Temp\ipykernel_31652\1692067836.py:4: S

# Step 3: Selection

In [21]:
def simple_binary_search_closest(arr: list, target: float) -> int:
    if not arr:
        return -1 # Return -1 for an empty list

    low = 0
    high = len(arr) - 1
    closest_idx = 0 # Initialize with first index as a default closest

    while low <= high:
        mid = (low + high) // 2
        mid_val = arr[mid]

        # If we found an exact match
        if mid_val == target:
            return mid

        # Update closest_idx if current mid_val is closer
        if abs(mid_val - target) < abs(arr[closest_idx] - target):
            closest_idx = mid
        elif abs(mid_val - target) == abs(arr[closest_idx] - target):
            # If equally close, prioritize smaller index or other criteria
            # For this simple version, we'll keep the one we already have unless explicitly told otherwise
            pass

        # Adjust search range
        if mid_val < target:
            low = mid + 1
        else:
            high = mid - 1

    # Compare `arr[high]` (value just less than target) and `arr[low]` (value just greater than target)
    if low >= len(arr): # Target is greater than all elements
        return len(arr) - 1
    if high < 0: # Target is less than all elements
        return 0

    val_at_low = arr[low]
    val_at_high = arr[high] # high will be one less than low if no exact match found

    diff_low = abs(val_at_low - target)
    diff_high = abs(val_at_high - target)

    if diff_low <= diff_high:
        return low
    else:
        return high


In [22]:
for data_type in ('local', 'non_local'):
    percent_per_score = 100.00 / etl_data[data_type]['score'].sum()
    etl_data[data_type]['percent_cut'] = etl_data[data_type]['score'] * percent_per_score

    etl_data[data_type] = etl_data[data_type].reset_index(drop = True)
    etl_data[data_type]['cum_sum'] = etl_data[data_type]['percent_cut'].cumsum()

C:\Users\alber\AppData\Local\Temp\ipykernel_31652\1249357792.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  etl_data[data_type]['percent_cut'] = etl_data[data_type]['score'] * percent_per_score
C:\Users\alber\AppData\Local\Temp\ipykernel_31652\1249357792.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  etl_data[data_type]['percent_cut'] = etl_data[data_type]['score'] * percent_per_score


In [23]:
todays_suggestions_log = pd.DataFrame()
for data_type in ('local', 'non_local'):
    random_selection = np.random.rand() * 100
    idx_selection = simple_binary_search_closest(etl_data[data_type]['cum_sum'].tolist(), random_selection)

    todays_suggestions_log = pd.concat([todays_suggestions_log, etl_data[data_type][etl_data[data_type].index == idx_selection]])

todays_suggestions_log['record_date'] = dt.datetime.today().date()
todays_suggestions_log

,connection_id,member_id,contact_id,local,most_recent_contact,type_of_contact,days_since_contact,number_of_contacts,norm_type_of_contact,norm_days_since_contact,norm_number_of_contacts,inv_norm_type_of_contact,inv_norm_number_of_contacts,score,percent_cut,cum_sum,record_date
12,80,1,80,1,2025-06-10,0,219,0,0.0,1.0,0.0,1.0,1.0,1.0,7.142857,78.571429,2026-01-15
60,71,1,71,0,2025-06-10,0,219,0,0.0,1.0,0.0,1.0,1.0,1.0,1.488834,88.089330,2026-01-15


In [24]:
todays_suggestions_log.to_csv(os.path.join(mart_database_folder, 'selections.csv'))

In [25]:
for idx, row in todays_suggestions_log.iterrows():
    if row['local']:
        print(f'local: {raw_db["member_contacts"][raw_db["member_contacts"]["contact_id"] == row["contact_id"]]["name"].iloc[0]}')
    else:
        print(f'non-local: {raw_db["member_contacts"][raw_db["member_contacts"]["contact_id"] == row["contact_id"]]["name"].iloc[0]}')

local: Nicole Chin-lyn
non-local: George Luo
